<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c359c9c6b9415719a64769402a618c3f500e7f3f0f7eaf7c38f01f64b6e6acb6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 11:10:16
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.42 C
-------------------
Today PnL: -32.77 K (-0.23%)
Current PnL: -26.76 L (-17.07%)
CY Booked + Current PnL: -12.47 L (-7.95%)
-------------------
Total profit:  1.83 L
Total loss:  -28.58 L
-------------------
Total Booked + Current PnL: 14.51 L (11.18%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.04%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 92.22 L (64.8%)
Deployed:  1.30 C
Current:  1.42 C
CAGR/XIRR %: 5.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.64,14.48,6.04,21.40,18354.0,38446.0,303872.0,6.20,60.0,X-LC,5.70,14.0,2.09,2.14,21.73,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,0.19,-21.25,27.10,0.09,21507.0,-21417.0,79360.0,84.20,36.0,M-SC,3.31,253.0,-1.00,0.56,4.26,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.79,-12.58,14.41,0.02,21991.0,-21952.0,152607.0,132.70,60.0,M-SC,8.11,240.0,-1.00,1.07,32.74,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.14,4.43,14.17,19.23,22264.0,6660.0,157120.0,22.81,73.0,L-LC,7.13,182.0,0.30,1.10,26.64,XY25,ATH,MINING
49,MASFIN,398.61,-1.62,-3.23,26.12,22.05,24766.0,-3165.0,94815.0,-16.44,48.0,H-SC,6.53,168.0,-0.13,0.67,37.82,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,216.00,3.18,-14.79,44.39,23.03,67736.0,-26479.0,152592.0,66.67,66.0,H-SC,4.21,165.0,-0.39,1.07,51.40,XR,NTT,BANKS
65,SIEMENS,4671.50,2.17,-17.39,51.93,25.51,79837.0,-32355.0,153740.0,-5.62,45.0,H-LC,1.73,50.0,-0.41,1.08,13.29,AR,ATH,ELECTRICAL
68,STARHEALTH,761.00,2.10,-11.56,66.50,47.26,165342.0,-32488.0,248635.0,20.72,47.0,H-SC,7.60,174.0,-0.20,1.75,33.21,XY24,NTT,INSURANCE
5,ASIANPAINT,3460.25,1.37,-1.27,22.50,20.95,55929.0,-3194.0,248574.0,-2.32,55.0,X-LC,12.25,36.0,-0.06,1.75,33.92,X40,ATH,PAINTS
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.00,0.59,8.03,OX40N,BTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,-3.60,-32.39,106.49,39.61,73798.0,-33200.0,69300.0,-54.30,51.0,H-SC,30.22,136.0,-0.45,0.49,12.17,AR,ATH,MISC
4,ALKYLAMINE,4546.37,-3.50,-33.93,193.09,93.63,128795.0,-34261.0,66702.0,-29.23,40.0,H-SC,16.66,143.0,-0.27,0.47,0.84,SR,ATH,CHEMICALS
76,TRIDENT,48.00,-3.47,-29.93,83.42,28.51,53923.0,-27614.0,64640.0,-17.17,38.0,M-SC,7.43,226.0,-0.51,0.45,10.61,XR,NTT,TEXTILES
61,ROUTE,2227.21,-2.88,-48.85,230.52,69.05,156890.0,-65007.0,68059.0,-58.67,42.0,H-SC,18.88,140.0,-0.41,0.48,4.96,SR,ATH,IT
28,GREENPANEL,537.00,-2.33,-41.03,142.71,43.14,154084.0,-75108.0,107970.0,141.60,32.0,M-SC,12.81,231.0,-0.49,0.76,1.56,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.22,0.32,20.35,20.74,41479.0,660.0,203830.0,33.49,58.0,M-MC,5.38,189.0,0.02,1.43,10.97,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.32,-2.07,112.31,107.91,162387.0,-3056.0,144588.0,-20.15,52.0,M-SC,10.11,220.0,-0.02,1.02,7.81,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.79,-12.58,14.41,0.02,21991.0,-21952.0,152607.0,132.70,60.0,M-SC,8.11,240.0,-1.00,1.07,32.74,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,1.11,-16.70,58.49,32.03,64234.0,-22015.0,109820.0,-9.82,67.0,H-SC,6.86,157.0,-0.34,0.77,23.01,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.19,-21.25,27.10,0.09,21507.0,-21417.0,79360.0,84.20,36.0,M-SC,3.31,253.0,-1.00,0.56,4.26,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.06,-21.75,45.01,13.47,94977.0,-58653.0,211014.0,-67.92,50.0,H-SC,4.57,159.0,-0.62,1.48,7.29,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-0.22,0.32,20.35,20.74,41479.0,660.0,203830.0,33.49,58.0,M-MC,5.38,189.0,0.02,1.43,10.97,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.14,4.43,14.17,19.23,22264.0,6660.0,157120.0,22.81,73.0,L-LC,7.13,182.0,0.30,1.10,26.64,XY25,ATH,MINING
84,WIPRO,420.00,0.44,8.01,59.48,72.27,108865.0,13580.0,183028.0,-7.44,51.0,M-LC,6.19,101.0,0.12,1.29,14.68,XR,NTT,IT
37,IEX,219.00,-0.65,1.20,46.86,48.63,94824.0,2402.0,202356.0,-32.69,61.0,H-SC,13.41,137.0,0.03,1.42,12.96,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.44,8.01,59.48,72.27,108865.0,13580.0,183028.0,-7.44,51.0,M-LC,6.19,101.0,0.12,1.29,14.68,XR,NTT,IT
37,IEX,219.00,-0.65,1.20,46.86,48.63,94824.0,2402.0,202356.0,-32.69,61.0,H-SC,13.41,137.0,0.03,1.42,12.96,XR,NTT,MISC
25,FINCABLES,1641.55,-0.32,-2.07,112.31,107.91,162387.0,-3056.0,144588.0,-20.15,52.0,M-SC,10.11,220.0,-0.02,1.02,7.81,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.56,-8.34,125.52,106.72,141905.0,-10282.0,113054.0,-54.63,33.0,H-SC,7.42,138.0,-0.07,0.79,13.09,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.26,-4.55,42.09,35.63,84196.0,-9536.0,200038.0,-18.07,40.0,H-MC,3.14,119.0,-0.11,1.41,11.36,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.69,-17.02,33.51,10.78,66821.0,-40912.0,199406.0,-50.55,12.0,X-LC,6.29,1.0,-0.61,1.40,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.20,-27.04,115.45,57.19,274703.0,-88186.0,237941.0,-65.54,25.0,X-MC,9.23,58.0,-0.32,1.67,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.32,-12.29,48.91,30.62,101699.0,-29124.0,207930.0,-37.71,28.0,X-MC,16.39,57.0,-0.29,1.46,0.00,X40,ATH,APPARELS
59,RELIANCE,1952.00,0.07,-5.54,32.64,25.30,61962.0,-11127.0,189836.0,-8.43,32.0,X-LC,8.07,26.0,-0.18,1.33,27.16,XY25,BTT,REFINERIES
78,UNITDSPR,1644.00,-0.32,-1.13,22.15,20.77,51276.0,-2649.0,231495.0,-9.72,33.0,X-MC,0.95,70.0,-0.05,1.63,5.71,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.32,-1.13,22.15,20.77,51276.0,-2649.0,231495.0,-9.72,33.0,X-MC,0.95,70.0,-0.05,1.63,5.71,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-0.20,-5.77,32.10,24.48,80872.0,-15428.0,251937.0,-7.34,43.0,X-LC,1.73,37.0,-0.19,1.77,46.03,XY24,BTT,FINANCE
65,SIEMENS,4671.50,2.17,-17.39,51.93,25.51,79837.0,-32355.0,153740.0,-5.62,45.0,H-LC,1.73,50.0,-0.41,1.08,13.29,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.25,1.81,41.14,43.70,103905.0,4496.0,252564.0,-6.50,65.0,X-MC,2.24,73.0,0.04,1.77,17.14,XY24,BTT,FMCG
81,VBL,671.64,0.05,1.23,34.02,35.67,108774.0,3892.0,319734.0,-10.77,63.0,X-LC,2.28,5.0,0.04,2.25,15.26,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.27,-27.77,127.25,64.15,218717.0,-66071.0,171880.0,-57.62,36.0,X-SC,6.25,84.0,-0.30,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-1.20,-27.04,115.45,57.19,274703.0,-88186.0,237941.0,-65.54,25.0,X-MC,9.23,58.0,-0.32,1.67,0.0,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.32,-12.29,48.91,30.62,101699.0,-29124.0,207930.0,-37.71,28.0,X-MC,16.39,57.0,-0.29,1.46,0.0,X40,ATH,APPARELS
43,ITC,452.00,-0.69,-17.02,33.51,10.78,66821.0,-40912.0,199406.0,-50.55,12.0,X-LC,6.29,1.0,-0.61,1.40,0.0,X40,NTT,FMCG
11,BATAINDIA,2096.00,0.20,-40.07,125.62,35.20,96862.0,-51563.0,77107.0,1.08,34.0,X-SC,17.32,93.0,-0.53,0.54,0.2,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.86,-27.89,98.10,42.84,45982.0,-18133.0,46873.0,-52.62,50.0,X-SC,27.03,83.0,-0.39,0.33,6.04,XY24,NTT,MISC
58,RELAXO,1176.00,-0.19,-48.31,194.63,52.31,146050.0,-70120.0,75040.0,-44.92,40.0,X-SC,7.82,92.0,-0.48,0.53,0.45,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.20,-40.07,125.62,35.20,96862.0,-51563.0,77107.0,1.08,34.0,X-SC,17.32,93.0,-0.53,0.54,0.20,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.77,1.05,22.79,24.08,29235.0,1331.0,128281.0,-7.09,53.0,X-SC,6.17,91.0,0.05,0.90,20.87,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.19,-17.32,73.35,43.32,98773.0,-28212.0,134660.0,-28.92,45.0,X-SC,7.85,90.0,-0.29,0.95,3.39,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.69,-17.02,33.51,10.78,66821.0,-40912.0,199406.0,-50.55,12.0,X-LC,6.29,1.0,-0.61,1.40,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.38,-26.60,67.55,22.98,111998.0,-60098.0,165800.0,-24.30,45.0,X-LC,4.09,2.0,-0.54,1.17,4.28,XY24,NTT,AUTO
73,TCS,4389.97,0.31,-11.63,36.59,20.71,122301.0,-43991.0,334246.0,-23.95,48.0,X-LC,2.49,3.0,-0.36,2.35,11.68,X40,ATH,IT
81,VBL,671.64,0.05,1.23,34.02,35.67,108774.0,3892.0,319734.0,-10.77,63.0,X-LC,2.28,5.0,0.04,2.25,15.26,X40N,ATH,FMCG
41,INFY,1972.00,0.66,10.55,21.44,34.24,74159.0,33003.0,345891.0,-13.85,52.0,X-LC,7.70,6.0,0.45,2.43,19.70,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.48,-34.47,102.78,32.89,52003.0,-26610.0,50596.0,-724.57,59.0,L-MC,7.75,259.0,-0.51,0.36,39.34,XR,NTT,BANKS
13,BSOFT,831.70,-0.19,-21.89,93.04,50.79,102220.0,-30786.0,109867.0,0.12,52.0,H-SC,10.72,171.0,-0.30,0.77,27.92,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.79,-12.58,14.41,0.02,21991.0,-21952.0,152607.0,132.70,60.0,M-SC,8.11,240.0,-1.00,1.07,32.74,OX40N,NTT,PAINTS
69,SURYODAY,216.00,3.18,-14.79,44.39,23.03,67736.0,-26479.0,152592.0,66.67,66.0,H-SC,4.21,165.0,-0.39,1.07,51.40,XR,NTT,BANKS
17,COALINDIA,484.83,-0.14,4.43,14.17,19.23,22264.0,6660.0,157120.0,22.81,73.0,L-LC,7.13,182.0,0.30,1.10,26.64,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.28,-29.05,59.13,12.90,91665.0,-63474.0,155022.0,128.37,64.0,H-SC,10.22,173.0,-0.69,1.09,28.90,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,1.11,-16.70,58.49,32.03,64234.0,-22015.0,109820.0,-9.82,67.0,H-SC,6.86,157.0,-0.34,0.77,23.01,OX40N,NTT,IT
17,COALINDIA,484.83,-0.14,4.43,14.17,19.23,22264.0,6660.0,157120.0,22.81,73.0,L-LC,7.13,182.0,0.30,1.10,26.64,XY25,ATH,MINING
69,SURYODAY,216.00,3.18,-14.79,44.39,23.03,67736.0,-26479.0,152592.0,66.67,66.0,H-SC,4.21,165.0,-0.39,1.07,51.40,XR,NTT,BANKS
36,ICICIPRULI,790.00,1.28,14.20,14.48,30.73,32576.0,27968.0,224973.0,-12.55,73.0,X-MC,6.64,79.0,0.86,1.58,28.78,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.31
2,50,76.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.86
MC    30.68
LC    26.49
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.37
X40      23.87
X40N     12.83
XY25     10.68
XR        9.09
AR        8.55
OX40N     7.69
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.28
X-LC    23.02
H-SC    22.95
M-SC    11.29
X-SC     7.83
H-MC     4.61
M-MC     1.43
M-LC     1.29
L-LC     1.10
H-LC     1.08
L-SC     0.79
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.03,-8.61,44.25
IT,12.88,-16.64,75.48
FINANCE,11.08,-13.18,60.68
MISC,7.13,-28.07,80.45
ELECTRICAL,5.92,-11.47,52.30
PAINTS,5.76,-12.54,29.26
INSURANCE,4.74,-0.60,35.03
PHARMA,3.92,-5.43,38.62
BANKS,2.80,-34.44,115.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3218768.0,21
AR,1355662.0,10
XR,1199909.0,12
X40,1121715.0,15
X40N,919771.0,9
OX40N,698168.0,10
XY25,421781.0,7
SR,285685.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3530449.0,24
X-MC,1578191.0,16
M-SC,1450937.0,15
X-LC,948219.0,13
X-SC,805666.0,8
H-MC,424259.0,3
L-SC,179290.0,2
M-LC,108865.0,1
H-LC,79837.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246919.0      6
           AR         923754.0      5
M-SC       XY24       837407.0      6
H-SC       XR         768423.0      7
X-MC       X40        510496.0      7
           XY24       445748.0      3
X-LC       X40        414935.0      6
X-MC       X40N       398864.0      4
X-SC       X40N       315448.0      3
H-SC       OX40N      305668.0      4
M-SC       OX40N      300188.0      5
X-SC       XY24       293934.0      3
H-SC       SR         285685.0      2
X-MC       XY25       223083.0      2
H-MC       AR         222369.0      2
X-LC       X40N       205459.0      2
H-MC       XY24       201890.0      1
X-SC       X40        196284.0      2
X-LC       XY24       192870.0      2
M-SC       XR         183640.0      2
X-LC       XY25       134955.0      3
M-SC       AR         129702.0      2
M-LC       XR         108865.0      1
L-SC       OX40N       92312.0      1
           XR          86978.0      1
H-LC       AR          79837.0      1
L-MC       XR          52003.0      1
M-MC       XY25        41479.0      1
L-LC       XY25        22264.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
